## Import Libraries


In [3]:
#!pip install pandas numpy matplotlib seaborn nltk textblob scikit-learn top2vec bertopic wordcloud

# !pip install top2vec
# !pip install bertopic
# !pip install gensim
# !pip install pyLDAvis

In [5]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re

# Text processing and NLP
from textblob import TextBlob
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Machine learning and topic modeling
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.decomposition import NMF
from top2vec import Top2Vec
from bertopic import BERTopic

# Visualization
from wordcloud import WordCloud


import gensim
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

ImportError: cannot import name 'triu' from 'scipy.linalg' (c:\Users\samue\switchdrive\SyncVM\MscIds Course Materials\4th Semester\CLT\CLT-Project\venv-sam-clt\Lib\site-packages\scipy\linalg\__init__.py)

## Load the Dataset

Load the preprocessed cleantech media dataset

In [6]:
# Load the Dataset

# Load the preprocessed cpatent media dataset
# Load the dataset
file_path = "patent_preprocessed.csv"
patent_data = pd.read_csv(file_path, encoding="utf-8")

## Dataset Overview

In [7]:
# Display the first few rows and the data structure to understand what we have
# Display the first few rows and the data structure to understand what we have

patent_data.info(), patent_data.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8545 entries, 0 to 8544
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   publication_number     8545 non-null   object
 1   application_number     8545 non-null   object
 2   country_code           8545 non-null   object
 3   publication_date       8545 non-null   object
 4   title                  8545 non-null   object
 5   abstract               8545 non-null   object
 6   inventors              8545 non-null   object
 7   code                   8545 non-null   object
 8   inventive              8545 non-null   bool  
 9   first                  8545 non-null   bool  
 10  title_preprocessed     8395 non-null   object
 11  abstract_preprocessed  4552 non-null   object
dtypes: bool(2), object(10)
memory usage: 684.4+ KB


(None,
   publication_number application_number country_code publication_date  \
 0   US-2022239235-A1  US-202217717397-A           US       2022-07-28   
 
                                                title  \
 0  Adaptable DC-AC Inverter Drive System and Oper...   
 
                                             abstract inventors        code  \
 0  Disclosed is an adaptable DC-AC inverter syste...        []  H02M7/5395   
 
    inventive  first                               title_preprocessed  \
 0       True  False  adaptable DC AC Inverter Drive System Operation   
 
                                abstract_preprocessed  
 0  Disclosed adaptable DC AC inverter system oper...  )

In [8]:
# Inspect the columns title_preprocessed and abstract_preprocessed for null values

patent_data.isnull().sum()

publication_number          0
application_number          0
country_code                0
publication_date            0
title                       0
abstract                    0
inventors                   0
code                        0
inventive                   0
first                       0
title_preprocessed        150
abstract_preprocessed    3993
dtype: int64

In [9]:
# Display the data dataframe head and tail

print(patent_data.head(2))

print(patent_data.tail(2))

# Filtering where abstract_preprocessed is null
filtered_data = patent_data[patent_data["abstract_preprocessed"].isnull()]

# Counting the occurrences of each country_code in the filtered data
country_code_counts = (
    filtered_data["country_code"].value_counts().sort_values(ascending=False)
)

# Display the counts
print(country_code_counts)

  publication_number application_number country_code publication_date  \
0   US-2022239235-A1  US-202217717397-A           US       2022-07-28   
1   US-2022239251-A1  US-202217580956-A           US       2022-07-28   

                                               title  \
0  Adaptable DC-AC Inverter Drive System and Oper...   
1  System for providing the energy from a single ...   

                                            abstract inventors        code  \
0  Disclosed is an adaptable DC-AC inverter syste...        []  H02M7/5395   
1  In accordance with an example embodiment, a so...        []   H02S40/38   

   inventive  first                                 title_preprocessed  \
0       True  False    adaptable DC AC Inverter Drive System Operation   
1       True  False  system provide energy single contiguous solar ...   

                               abstract_preprocessed  
0  Disclosed adaptable DC AC inverter system oper...  
1  accordance example embodiment solar ener

### NLP Preprocessing Issue Resolution Report

**Background:**  
Our NLP framework encountered issues in effectively preprocessing multilingual patent data, particularly for Chinese (CN), Taiwanese (TW), and World Intellectual Property Organization (WO) codes, resulting in a substantial number of unprocessed entries.

**Challenges Identified:**  
- **Model Limitations:** The spaCy Chinese model (`zh_core_web_sm`) may not fully capture the nuances of domain-specific Chinese used in patents, impacting the effectiveness of tokenization, lemmatization, and stop word removal.
- **Data Quality Issues:** Potential inaccuracies in language tagging and text quality could be leading to preprocessing failures.
- **Improper Text Handling:** The function may not handle specific cases well, leading to failures in text processing.
- **Evaluation Methods:** Current metrics may not accurately reflect the preprocessing effectiveness.

**Strategic Decision:**
- Despite these challenges, the decision has been made to not focus further on these issues. We will continue with our project's next steps and shift our resources towards progressing in other strategic areas.